Conectando ao Google Drive, se necessário. Ou então rodar local. Descomentar os caminhos nas células.

In [ ]:
environment = "Google Drive"
#environment = "Local"

In [ ]:
import os
import pandas as pd

if environment = "Google Drive":
    from google.colab import drive
    drive.mount('/content/drive')


Pre - Processamento

In [1]:
import json
import re
import os
from tqdm import tqdm

# Definir o ambiente e os caminhos
environment = "Local" 
environment = "Google Drive" # Altere para "Google Drive" se estiver usando o Google Drive

if environment == "Google Drive":
    # Caminhos para o Google Drive
    input_path = '/content/drive/MyDrive/ADAN/ADAN_notebooks/DBLP/dataset/dblp.json'
    output_dir = '/content/drive/MyDrive/ADAN/ADAN_notebooks/DBLP/data/'
else:
    # Caminhos locais
    input_path = '/home/neiruto/Documents/ueg/andsol/adan/repositories/DBLP/dataset/dblp.json'
    output_dir = '/home/neiruto/Documents/ueg/andsol/adan/repositories/DBLP/data/'

# Verificar e criar o diretório de saída se ele não existir
os.makedirs(output_dir, exist_ok=True)

# Dicionários para armazenar os dados
papers = {}
authors = {}
venues = {}
word = {}

# Inicializar IDs
keyid = 0

# Expressão regular para limpar texto
r = '[!“”"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~—～]+'
stopword = ['at', 'based', 'in', 'of', 'for', 'on', 'and', 'to', 'an', 'using', 'with', 'the']

# Abrir arquivos de saída
f1 = open(os.path.join(output_dir, 'paper_author.txt'), 'w', encoding='utf-8')
f2 = open(os.path.join(output_dir, 'paper_venue.txt'), 'w', encoding='utf-8')
f3 = open(os.path.join(output_dir, 'paper_word.txt'), 'w', encoding='utf-8')
f4 = open(os.path.join(output_dir, 'paper_title.txt'), 'w', encoding='utf-8')
f5 = open(os.path.join(output_dir, 'paper_author_names.txt'), 'w', encoding='utf-8')  # Arquivo para nomes de autores
f6 = open(os.path.join(output_dir, 'paper_abstract.txt'), 'w', encoding='utf-8')  # Arquivo para abstracts
f7 = open(os.path.join(output_dir, 'paper_venue_name.txt'), 'w', encoding='utf-8')  # Novo arquivo para venues e nomes

# Carrega os dados do arquivo JSON
with open(input_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# Processar o JSON com uma barra de progresso
with tqdm(total=len(data), desc="Processando JSON") as pbar:
    for entry in data:
        # Verificar se o ID é numérico e se o label é um inteiro
        pid = entry.get('id')
        label = entry.get('label')

        # Verifica se 'id' e 'label' são inteiros
        if not isinstance(pid, int) or not isinstance(label, int):
            pbar.update(1)
            continue

        title = str(entry.get('title', '')).strip()
        author = str(entry.get('author', '')).strip()
        coauthors = entry.get('coauthors', []) if entry.get('coauthors') else []
        abstract = str(entry.get('abstract', '')).strip()
        venue = str(entry.get('venue', '')).strip()  # Adicionado para o processamento do venue

        # Verificar se o abstract não está vazio
        if abstract:
            f6.write(f'i{pid}\t{abstract}\n')

        # Processar autor principal e coautores
        all_authors = [author] + coauthors
        for author_name in all_authors:
            author_clean = author_name.replace(" ", "")
            if author_clean not in authors:
                authors[author_clean] = keyid
                keyid += 1
            f1.write(f'i{pid}\t{authors[author_clean]}\n')

        # Relacionar o paper_id com os nomes dos autores
        f5.write(f'i{pid}\t{",".join(all_authors)}\n')

        # Processar venues
        if venue not in venues:
            venues[venue] = keyid
            keyid += 1
        f2.write(f'i{pid}\t{venues[venue]}\n')

        # Processar título
        title_cleaned = re.sub(r, ' ', title)
        title_cleaned = title_cleaned.replace('\t', ' ')
        title_cleaned = title_cleaned.lower()

        f4.write(f'i{pid}\t{title_cleaned}\n')

        # Processar o arquivo de venue com o nome do paper e o nome da venue
        f7.write(f'i{pid}\t{venue}\n')

        # Processar palavras do título e contar ocorrências
        split_cut = title_cleaned.split(' ')
        for word_part in split_cut:
            if word_part and word_part not in stopword:
                if word_part in word:
                    word[word_part] += 1
                else:
                    word[word_part] = 1

        pbar.update(1)  # Atualizando a barra de progresso para cada entrada processada

# Agora que todas as palavras foram contadas, escrevemos aquelas que aparecem 2 ou mais vezes
with tqdm(total=len(data), desc="Escrevendo palavras") as pbar:
    for entry in data:
        pid = entry.get('id')
        title = str(entry.get('title', '')).strip()

        # Processar título
        title_cleaned = re.sub(r, ' ', title)
        title_cleaned = title_cleaned.replace('\t', ' ')
        title_cleaned = title_cleaned.lower()

        # Processar palavras do título e escrever se aparecerem mais de 2 vezes
        split_cut = title_cleaned.split(' ')
        for word_part in split_cut:
            if word_part in word and word[word_part] >= 2:
                f3.write(f'i{pid}\t{word_part}\n')

        pbar.update(1)  # Atualizando a barra de progresso para cada entrada processada

# Fechar arquivos de saída
f1.close()
f2.close()
f3.close()
f4.close()
f5.close()
f6.close()
f7.close()

print('Processamento concluído.')


Escrevendo palavras: 100%|██████████| 6783/6783 [00:00<00:00, 42466.63it/s]

Processamento concluído.


Geração da rede heterogênea.

In [3]:
import os
import pickle
import networkx as nx

class HeterogeneousNetworkGenerator:
    def __init__(self):
        self.G = nx.Graph()  # Rede NetworkX

    def read_data(self, dirpath):
        # Carregar e processar arquivos de relacionamentos

        # Documentos e Palavras
        with open(os.path.join(dirpath, "paper_word.txt"), encoding='utf-8') as file:
            for line in file:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    paper_id, word = toks
                    self.G.add_node(paper_id, type='paper')
                    self.G.add_node(word, type='word')
                    self.G.add_edge(paper_id, word, relationship='contains')

        # Documentos e Autores
        with open(os.path.join(dirpath, "paper_author.txt"), encoding='utf-8') as file:
            for line in file:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    paper_id, author_id = toks
                    self.G.add_node(paper_id, type='paper')
                    self.G.add_node(author_id, type='author')
                    self.G.add_edge(paper_id, author_id, relationship='written_by')

        # Documentos e Conferências
        with open(os.path.join(dirpath, "paper_venue.txt"), encoding='utf-8') as file:
            for line in file:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    paper_id, conf_id = toks
                    self.G.add_node(paper_id, type='paper')
                    self.G.add_node(conf_id, type='conference')
                    self.G.add_edge(paper_id, conf_id, relationship='presented_at')

        # Documentos e Títulos
        with open(os.path.join(dirpath, "paper_title.txt"), encoding='utf-8') as file:
            for line in file:
                toks = line.strip().split("\t", 1)
                if len(toks) == 2:
                    paper_id, title = toks
                    title_node_id = f'title_{paper_id}'

                    self.G.add_node(title_node_id, type='title', content=title)
                    self.G.add_edge(paper_id, title_node_id, relationship='has_title')

        # Documentos e Abstracts
        with open(os.path.join(dirpath, "paper_abstract.txt"), encoding='utf-8') as file:
            for line in file:
                toks = line.strip().split("\t", 1)
                if len(toks) == 2:
                    paper_id, abstract = toks
                    abstract_node_id = f'abstract_{paper_id}'

                    self.G.add_node(abstract_node_id, type='abstract', content=abstract)
                    self.G.add_edge(paper_id, abstract_node_id, relationship='has_abstract')

        # Documentos e Nomes de Autores
        with open(os.path.join(dirpath, "paper_author_names.txt"), encoding='utf-8') as file:
            for line in file:
                toks = line.strip().split("\t", 1)
                if len(toks) == 2:
                    paper_id, author_names = toks
                    author_names_node_id = f'author_names_{paper_id}'

                    self.G.add_node(author_names_node_id, type='author_names', content=author_names)
                    self.G.add_edge(paper_id, author_names_node_id, relationship='has_author_names')

        # Documentos e Organizações (se aplicável)
        if os.path.exists(os.path.join(dirpath, "paper_organization.txt")):
            with open(os.path.join(dirpath, "paper_organization.txt"), encoding='utf-8') as file:
                for line in file:
                    toks = line.strip().split("\t")
                    if len(toks) == 2:
                        paper_id, organization = toks
                        org_node_id = f'org_{organization}'

                        self.G.add_node(org_node_id, type='organization', name=organization)
                        self.G.add_edge(paper_id, org_node_id, relationship='affiliated_with')

    def save_network(self, filepath):
        # Salvar o grafo para uso futuro
        with open(filepath, 'wb') as file:
            pickle.dump(self.G, file)
        print("Rede heterogênea salva com sucesso.")

def main():

    if environment == "Google Drive":
        #Google Drive
        dirpath = "/content/drive/MyDrive/ADAN/ADAN_notebooks/DBLP/data"

    else:
        #Local
        dirpath = "/home/neiruto/Documents/ueg/andsol/adan/repositories/DBLP/data/"

    filepath = os.path.join(dirpath, "HeterogeneousNetwork.pkl")


    hng = HeterogeneousNetworkGenerator()
    hng.read_data(dirpath)
    hng.save_network(filepath)

if __name__ == "__main__":
    main()


Rede heterogênea salva com sucesso.


In [4]:
import torch
from transformers import AutoTokenizer, AutoModel
import pickle
import os
from tqdm import tqdm
import numpy as np

environment = "Google Drive"

if environment == "Google Drive":

    # Diretórios e arquivos (Google Drive)
    dirpath = "/content/drive/MyDrive/ADAN/ADAN_notebooks/DBLP/"
    data_dir = os.path.join(dirpath, 'data/')
    model_dir = "/content/drive/MyDrive/ADAN/fine_tuning_sci_bert/output22"  # Diretório onde está o seu modelo fine-tunado

else:
    # Diretórios e arquivos (Local)
    dirpath = "/home/neiruto/Documents/ueg/andsol/adan/repositories/DBLP/"
    data_dir = os.path.join(dirpath, 'data/')
    model_dir = "/home/neiruto/Documents/ueg/andsol/adan/scibert/scibert_model/output22"  # Diretório onde está o seu modelo fine-tunado


# Função auxiliar para substituir 'null' ou None por um ponto (.) ou string vazia
def clean_value(value, substitute='.'):
    if value is None or str(value).lower() == 'null':  # Tratar casos de None e string 'null'
        return substitute
    return str(value).strip()

# Carregar as features com base nos parâmetros escolhidos
def load_features(features_to_include):
    paperid_abstract = {}
    paperid_author_names = {}
    paperid_title = {}
    paperid_venue_name = {}
    paperid_words = {}

    # Carregar abstracts
    if 'abstract' in features_to_include:
        with open(os.path.join(data_dir, "paper_abstract.txt"), encoding='utf-8') as abstract_file:
            for line in abstract_file:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    paperid_abstract[toks[0]] = clean_value(toks[1], '.')

    # Carregar nomes dos autores
    if 'author_names' in features_to_include:
        with open(os.path.join(data_dir, "paper_author_names.txt"), encoding='utf-8') as author_file:
            for line in author_file:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    paperid_author_names[toks[0]] = clean_value(toks[1], ' ')

    # Carregar títulos
    if 'title' in features_to_include:
        with open(os.path.join(data_dir, "paper_title.txt"), encoding='utf-8') as title_file:
            for line in title_file:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    paperid_title[toks[0]] = clean_value(toks[1], '.')

    # Carregar nomes de venues
    if 'venue_name' in features_to_include:
        with open(os.path.join(data_dir, "paper_venue_name.txt"), encoding='utf-8') as venue_file:
            for line in venue_file:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    paperid_venue_name[toks[0]] = clean_value(toks[1], ' ')

    # Carregar palavras
    if 'word' in features_to_include:
        with open(os.path.join(data_dir, "paper_word.txt"), encoding='utf-8') as words_file:
            for line in words_file:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    if toks[0] not in paperid_words:
                        paperid_words[toks[0]] = []
                    paperid_words[toks[0]].append(clean_value(toks[1], ' '))

    return paperid_abstract, paperid_author_names, paperid_title, paperid_venue_name, paperid_words

# Função para combinar as features selecionadas
def combine_features(paperid_abstract, paperid_author_names, paperid_title, paperid_venue_name, paperid_words, features_to_include):
    documents = []
    paper_ids = []
    for paperid in paperid_title:
        abstract = clean_value(paperid_abstract.get(paperid, '.'), ' ') if 'abstract' in features_to_include else ''
        author_names = clean_value(paperid_author_names.get(paperid, ""), ' ') if 'author_names' in features_to_include else ''
        title = clean_value(paperid_title.get(paperid, '.'), ' ') if 'title' in features_to_include else ''
        venue_name = clean_value(paperid_venue_name.get(paperid, ""), ' ') if 'venue_name' in features_to_include else ''
        words = " ".join([clean_value(word, '.') for word in paperid_words.get(paperid, [])]) if 'word' in features_to_include else ''

        combined_text = f"{abstract} {author_names} {title} {venue_name} {words}".strip()

        if combined_text and not combined_text.isspace():  # Garantir que o documento não está vazio
            documents.append(combined_text)
            paper_ids.append(paperid)
        else:
            print(f"Warning: Empty or invalid document for paper ID {paperid}")

    return documents, paper_ids

# Carregar o modelo BERT fine-tunado e o tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModel.from_pretrained(model_dir)

# Função para obter embeddings usando o modelo fine-tunado
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # Usar a média dos embeddings da última camada
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
    return embedding

# Função principal para extrair embeddings
def extract_embeddings(features_to_include):
    # Carregar features selecionadas
    paperid_abstract, paperid_author_names, paperid_title, paperid_venue_name, paperid_words = load_features(features_to_include)

    # Combinar features
    documents, paper_ids = combine_features(paperid_abstract, paperid_author_names, paperid_title, paperid_venue_name, paperid_words, features_to_include)

    # Extrair embeddings para todos os documentos
    paper_vec = {}
    for i, doc in enumerate(tqdm(documents, desc="Extraindo embeddings")):
        paper_vec[paper_ids[i]] = get_embedding(doc)

    # Verificar se há vetores zero
    zero_vector_count = 0
    for paper_id, embedding in paper_vec.items():
        if np.all(embedding == 0):
            print(f"Warning: Embedding for paper ID {paper_id} is a zero vector.")
            zero_vector_count += 1

    print(f"Total zero vectors found: {zero_vector_count}")

    # Salvar os embeddings
    with open(os.path.join(data_dir, 'scibert_emb.pkl'), "wb") as file_obj:
        pickle.dump(paper_vec, file_obj)

    print("Extração de embeddings concluída e embeddings salvos.")

# Parâmetros de features que podem ser incluídas: 'abstract', 'author_names', 'title', 'venue_name', 'word'
features_to_include = ['abstract', 'author_names', 'title', 'venue_name', 'word']  # Exemplo de como selecionar as features
extract_embeddings(features_to_include)


/home/neiruto/miniconda3/envs/adan/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Extraindo embeddings:   3%|▎         | 190/6783 [09:05<5:15:18,  2.87s/it] 


KeyboardInterrupt: 

GCN

In [ ]:
!pip install torch
!pip install torch-geometric
#!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.9.0+cu111.html
#!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.9.0+cu111.html
#!pip install torch-cluster -f https://data.pyg.org/whl/torch-1.9.0+cu111.html
#!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-1.9.0+cu111.html
#!pip install torch-geometric

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import numpy as np
import pickle
import os

# Verificar se a GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando o dispositivo: {device}")

# Diretórios e arquivos
dirpath = "/content/drive/MyDrive/ADAN/ADAN_notebooks/DBLP/"
data_dir = os.path.join(dirpath, 'data/')

# Carregar a rede heterogênea
with open(os.path.join(data_dir, "HeterogeneousNetwork.pkl"), 'rb') as file:
    G = pickle.load(file)

# Função para carregar embeddings
def load_embeddings(embedding_type):
    try:
        with open(os.path.join(data_dir, f'{embedding_type}_emb.pkl'), "rb") as file_obj:
            return pickle.load(file_obj)
    except FileNotFoundError:
        print(f"Erro: Arquivo de embeddings {embedding_type} não encontrado.")
        return None

def prepare_features(embedding_type, G, embeddings):
    nodes = list(G.nodes)
    node_idx_map = {node: idx for idx, node in enumerate(nodes)}
    edges = [(node_idx_map[u], node_idx_map[v]) for u, v in G.edges]

    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous().to(device)

    if embeddings:
        sample_embedding = next(iter(embeddings.values()))
        embedding_dim = sample_embedding.shape[0]
        features = []
        for node in nodes:
            features.append(embeddings.get(node, np.zeros(embedding_dim)))
    else:
        embedding_dim = 128
        features = np.random.normal(loc=0.0, scale=1.0, size=(len(nodes), embedding_dim))

    features = np.array(features)
    x = torch.tensor(features, dtype=torch.float).to(device)

    return Data(x=x, edge_index=edge_index), nodes, embedding_dim

# Definir o modelo GCN com quantidade de camadas variável
class GCN(torch.nn.Module):
    def __init__(self, input_dim, num_layers):
        super(GCN, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(input_dim, 512))

        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(512, 512))

        self.convs.append(GCNConv(512, 512))
        self.fc = torch.nn.Linear(512, input_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        for conv in self.convs:
            x = conv(x, edge_index)
            x = F.relu(x)
        x = self.fc(x)
        return x

def train_gcn(data, input_dim, num_layers, epochs=100):
    model = GCN(input_dim, num_layers).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

    def train():
        model.train()
        optimizer.zero_grad()
        out = model(data)
        loss = F.mse_loss(out, data.x)
        loss.backward()
        optimizer.step()
        return loss

    for epoch in range(epochs):
        loss = train()
        if epoch % 500 == 0 or epoch == epochs - 1:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

    model.eval()
    with torch.no_grad():
        new_embeddings = model(data).cpu().numpy()

    return new_embeddings

def save_embeddings(embeddings, nodes, filename):
    embeddings_dict = {node: embeddings[idx] for idx, node in enumerate(nodes)}
    with open(filename, "wb") as file_obj:
        pickle.dump(embeddings_dict, file_obj)

# Configurações de experimentos
configurations = [
    #{"embedding": "tfidf", "description": "GCN with TF-IDF embeddings", "output_dim": 512, "num_layers": 3},
    #{"embedding": "word2vec", "description": "GCN with Word2Vec embeddings", "output_dim": 100, "num_layers": 3},
    #{"embedding": "random", "description": "GCN with Random embeddings", "output_dim": 100, "num_layers": 3},
    {"embedding": "scibert", "description": "GCN with SciBERT embeddings", "output_dim": 768, "num_layers": 3}
]

# Parâmetros comuns
epochs = 10000

# Rodar experimentos
for config in configurations:
    embeddings = load_embeddings(config['embedding'])
    if embeddings is None:
        continue
    data, nodes, input_dim = prepare_features(config['embedding'], G, embeddings)
    new_embeddings = train_gcn(data, input_dim=input_dim, num_layers=config["num_layers"], epochs=epochs)

    save_path = os.path.join(dirpath, f'final_emb/L3-10000/pemb_final_gcn_{config["description"].replace(" ", "_").lower()}.pkl')
    save_embeddings(new_embeddings, nodes, save_path)

    print(f"{config['description']} completed and embeddings saved at {save_path}.")


Pré - GHAC

In [ ]:
import json
import os
import re

raw_path = "/content/drive/MyDrive/ADAN/ADAN_notebooks/DBLP/dataset/"

# Carregar o JSON principal
json_file = raw_path + 'dataset_formatted.json'  # Substitua pelo nome do seu arquivo JSON
with open(json_file, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Criar um diretório para armazenar os JSONs de cada autor
output_dir = raw_path + 'autores_json/'
os.makedirs(output_dir, exist_ok=True)

# Função para limpar o nome do autor para ser usado como nome de arquivo
def clean_filename(name):
    if not name or not isinstance(name, str):
        return "unknown_author"
    # Substituir caracteres inválidos por underscore
    return re.sub(r'[\/:*?"<>|]', '_', name.replace(' ', '_'))

# Separar os dados por autor e salvar em JSONs individuais
for entry in data:
    author = entry.get('author')

    # Verificar se o campo 'author' existe e é uma string válida
    if author is None or not isinstance(author, str) or author.strip() == '':
        author = 'unknown_author'
    else:
        author = author.strip()

    author_filename = clean_filename(author) + '.json'
    json_output = os.path.join(output_dir, author_filename)

    # Se o arquivo já existe, adiciona a entrada ao arquivo existente
    if os.path.isfile(json_output):
        with open(json_output, 'r', encoding='utf-8') as file:
            existing_data = json.load(file)

        existing_data.append(entry)

        with open(json_output, 'w', encoding='utf-8') as file:
            json.dump(existing_data, file, indent=4, ensure_ascii=False)
    else:
        with open(json_output, 'w', encoding='utf-8') as file:
            json.dump([entry], file, indent=4, ensure_ascii=False)

print("JSONs criados para cada nome ambíguo de autor.")
